In [ ]:
from icecube import dataio, icetray , dataclasses 


In [ ]:
import pandas as pd
pd.set_option("display.max_columns", None)   # tüm kolonlar


In [ ]:
from collections.abc import Iterable

In [ ]:
GCD_PATH = "/project/6008051/pone_simulation/GCD_Library/PONE_800mGrid.i3.gz"
data_file = dataio.I3File(GCD_PATH)
data_file.pop_frame()
calibration = data_file.pop_frame()

In [ ]:
# dir(calibration)

In [ ]:
print(type(calibration))
print("Stop:", calibration.Stop)   
print("Key's:", list(calibration.keys()))


In [ ]:
# Small helper: print the type and a short summary for each key in the frame
for key in calibration.keys():
    obj = calibration[key]
    print("="*60)
    print(f"Key: {key}")
    print("Type:", type(obj))
    try:
        print("dir(obj) first 10:", dir(obj)[:10])
    except Exception as e:
        print("dir(obj) could not be obtained:", e)

# 2.1 Calibration.I3OMGeoMap 
##### I suspect this is same as Geometry.I3OMGeoMap. Check it

# 2.2 Calibration.I3ModuleGeoMap
######  I suspect this is same as Geometry.I3ModuleGeoMap. Check it

# 2.3 Calibration.Subdetectors
##### I suspect this is same as Geometry.Subdetectors. Check it

# 2.4 Calibration.I3Geometry
##### I suspect this is same as Geometry.I3Geometry. Check it

# 2.5 Calibration.StartTime and Calibration.EndTime
##### I suspect these are same as Geometry.StartTime and Geometry.EndTime. Check it

# 2.6 Calibration.I3Calibration: pfff did not understand

In [ ]:
cal_obj = calibration["I3Calibration"]

print("Type:", type(cal_obj))



In [ ]:

# --- list public attributes ---
attrs = [a for a in dir(cal_obj) if not a.startswith("_")]
print("\nPublic attrs:")
for a in attrs:
    print("  -", a)



In [ ]:

rows = []
for a in attrs:
    val = getattr(cal_obj, a)
    try:
        n = len(val)
    except TypeError:
        n = None

    rows.append({
        "attr": a,
        "type": type(val).__name__,
        "len": n,
        "repr_sample": repr(val)[:80],   # first 80 chars
    })

cal_i3calib_df = pd.DataFrame(rows)
cal_i3calib_df

### 2.6.1 dom_cal

In [ ]:
dom_cal = cal_obj.dom_cal
print("Type:", type(dom_cal))
print("Number of entries:", len(dom_cal))


In [ ]:

# peek at one example DOM calibration
first_key, first_val = next(iter(dom_cal.items()))
print("\nExample key:", first_key)
print("Example value type:", type(first_val))
print("Example value repr:\n", first_val)



In [ ]:
import pandas as pd
import inspect

def extract_fields(obj, prefix="", depth=0, max_depth=4):
    """
    Recursively collect fields from an IceCube dataclass into a flat table.
    """
    rows = []

    if depth > max_depth:
        return rows

    for name in dir(obj):
        if name.startswith("_"):
            continue

        try:
            v = getattr(obj, name)
        except Exception:
            continue

        # skip callables (methods, functions)
        if inspect.ismethod(v) or inspect.isfunction(v):
            continue

        path = f"{prefix}{name}" if prefix == "" else f"{prefix}.{name}"

        # simple scalar → one row
        if isinstance(v, (int, float, bool, str)):
            rows.append({
                "field": path,
                "type": type(v).__name__,
                "value": v,
            })

        # everything else → summary row + recurse into it
        else:
            rows.append({
                "field": path,
                "type": type(v).__name__,
                "value": repr(v),
            })
            rows.extend(extract_fields(v, prefix=path, depth=depth+1, max_depth=max_depth))

    return rows


In [ ]:
# first_val is your I3DOMCalibration object
rows = extract_fields(first_val)

dom_full_df = pd.DataFrame(rows).sort_values("field")
dom_full_df


In [ ]:
# take one example DOM calibration
example_cal = first_val   # from your previous cell
# or: example_cal = next(iter(dom_cal.values()))

scalar_attrs = []
for a in dir(example_cal):
    if a.startswith("_"):
        continue
    v = getattr(example_cal, a)

    # treat these as "simple" fields
    if isinstance(v, (int, float, bool, str)):
        scalar_attrs.append(a)
    elif type(v).__name__ == "I3Time":
        scalar_attrs.append(a)

print("Scalar-ish attributes we will put into the DataFrame:")
for a in scalar_attrs:
    print("  -", a)


In [ ]:
rows = []

for omkey, calib in dom_cal.items():
    row = {
        "string": omkey.string,
        "om": omkey.om,
        "pmt": getattr(omkey, "pmt", None),
    }

    for a in scalar_attrs:
        v = getattr(calib, a)

        # convert I3Time to a readable string (or DAQ time if you prefer)
        if type(v).__name__ == "I3Time":
            v = repr(v)  # or v.utc_daq_time if that exists

        row[a] = v

    rows.append(row)

dom_cal_df = pd.DataFrame(rows)
dom_cal_df.head()


In [ ]:

# inspect attributes of a single DOM calibration object
dom_attrs = [a for a in dir(first_val) if not a.startswith("_")]

rows = []
for a in dom_attrs:
    v = getattr(first_val, a)
    try:
        n = len(v)
    except TypeError:
        n = None
    rows.append({"attr": a, "type": type(v).__name__, "len": n, "repr": repr(v)[:80]})

dom_cal_attrs_df = pd.DataFrame(rows)
dom_cal_attrs_df

vem_cal'a da bak

# 2.7 Calibration.SPEAbove
##### same for all OMs??

In [ ]:

spe_above = calibration["SPEAbove"]

print("Type:", type(spe_above))



In [ ]:
try:
    print("Number of entries:", len(spe_above))
except TypeError:
    print("len() not defined")

# peek at a few entries
print("\nFirst few entries:")
for i, (k, v) in enumerate(spe_above.items()):
    print("  key:", k, "   value:", v)
    if i >= 4:
        break

In [ ]:
print(spe_above)

In [ ]:
if len(spe_above) == 1:
    (only_key, only_val), = spe_above.items()
    print("Single entry in SPEAbove:")
    print("  key  :", only_key)
    print("  value:", only_val)
else:
    print("SPEAbove has", len(spe_above), "entries.")


spe_above_df = pd.DataFrame(
    [{"string": only_key.string,
      "om": only_key.om,
      "pmt": getattr(only_key, "pmt", None),
      "value": only_val}]
)
spe_above_df


# 2.8 Calibration.SPEScalingFactors

In [ ]:
# 2.8 Calibration.SPEScalingFactors

spe_scale = calibration["SPEScalingFactors"]

print("Type:", type(spe_scale))



In [ ]:
try:
    print("Number of entries:", len(spe_scale))
except TypeError:
    print("len() not defined")

# ilk birkaçını görelim
print("\nFirst few entries:")
for i, (k, v) in enumerate(spe_scale.items()):
    print("  key:", k, "   value:", v)
    if i >= 4:
        break

In [ ]:

rows = []
for k, v in spe_scale.items():
    rows.append({
        "string": getattr(k, "string", None),
        "om": getattr(k, "om", None),
        "pmt": getattr(k, "pmt", None),
        "key_repr": str(k),
        "value": v,
    })

spe_scale_df = pd.DataFrame(rows)
spe_scale_df.head()
